In [ ]:
# short version of https://github.com/rhiever/tpot/blob/master/tutorials/Titanic_Kaggle.ipynb

import pandas as pd
import numpy as np
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

data = pd.read_csv('./data/raw/train.csv')

# TPOT requires target to be called class
data.rename(columns={'Survived': 'class'}, inplace=True)

# TPOT requires all data to be numeric
data['Sex'] = data['Sex'].map({'male':0,'female':1})
data['Embarked'] = data['Embarked'].map({'S':0,'C':1,'Q':2})
data = data.fillna(-999)
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
cabin_transformed = mlb.fit_transform([{str(val)} for val in data['Cabin'].values])
assert (len(data['Cabin'].unique()) == len(mlb.classes_)), "Not Equal"

# drop categorical column with many labels
data_new = data.drop(['Name','Ticket','Cabin','class'], axis=1)

data_new = np.hstack((data_new.values, cabin_transformed))
data_class = data['class'].values

In [ ]:
training_indices, testing_indices = train_test_split(data.index, stratify=data_class, train_size=0.8, test_size=0.2)
tpot = TPOTClassifier(verbosity=2, max_time_mins=2, max_eval_time_mins=0.04, population_size=40)
tpot.fit(data_new[training_indices], data_class[training_indices])

In [ ]:
predictions = tpot.predict(data_new[testing_indices])
(predictions == data_class[testing_indices]).sum() / predictions.shape[0]